In [2]:
!git clone https://github.com/Karan-Malik/FaceMaskDetector.git

'git' is not recognized as an internal or external command,
operable program or batch file.


In [58]:
cd FaceMaskDetector

C:\Users\Zohaib\FaceMaskDetector


In [59]:
pip install -r requirements.txt

^C
Note: you may need to restart the kernel to use updated packages.


In [60]:
import numpy as np
import keras
import keras.backend as k
from keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from keras.models import Sequential,load_model
# from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from keras.preprocessing import image
import cv2
import datetime

In [61]:
# UNCOMMENT THE FOLLOWING CODE TO TRAIN THE CNN FROM SCRATCH

# BUILDING MODEL TO CLASSIFY BETWEEN MASK AND NO MASK

model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [62]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [63]:

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        'train',
        target_size=(150,150),
        batch_size=16 ,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        'test',
        target_size=(150,150),
        batch_size=16,
        class_mode='binary')

model_saved=model.fit_generator(
        training_set,
        epochs=10,
        validation_data=test_set,

        )


Found 1314 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


C:\Users\Zohaib\AppData\Local\Temp/ipykernel_11224/3747508810.py:15: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_saved=model.fit_generator(


Epoch 1/10
83/83 [==============================] - 86s 933ms/step - loss: 0.3408 - accuracy: 0.8508 - val_loss: 0.1223 - val_accuracy: 0.9588
Epoch 2/10
83/83 [==============================] - 47s 568ms/step - loss: 0.1554 - accuracy: 0.9429 - val_loss: 0.1207 - val_accuracy: 0.9588
Epoch 3/10
83/83 [==============================] - 49s 593ms/step - loss: 0.1174 - accuracy: 0.9574 - val_loss: 0.1032 - val_accuracy: 0.9691
Epoch 4/10
83/83 [==============================] - 48s 570ms/step - loss: 0.1066 - accuracy: 0.9619 - val_loss: 0.1171 - val_accuracy: 0.9691
Epoch 5/10
83/83 [==============================] - 47s 563ms/step - loss: 0.0799 - accuracy: 0.9741 - val_loss: 0.1274 - val_accuracy: 0.9536
Epoch 6/10
83/83 [==============================] - 47s 570ms/step - loss: 0.1260 - accuracy: 0.9566 - val_loss: 0.0912 - val_accuracy: 0.9845
Epoch 7/10
83/83 [==============================] - 48s 571ms/step - loss: 0.0702 - accuracy: 0.9756 - val_loss: 0.0981 - val_accuracy: 0.9691

In [67]:

mymodel=load_model('mymodel.h5')
#test_image=image.load_img('C:/Users/Karan/Desktop/ML Datasets/Face Mask Detection/Dataset/test/without_mask/30.jpg',target_size=(150,150,3))
test_image=image.load_img(r'C:/Users/Zohaib/Desktop/0-with-mask.jpg',
                          target_size=(150,150,3))
test_image
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
mymodel.predict(test_image)[0][0]

0.0

In [ ]:

# IMPLEMENTING LIVE DETECTION OF FACE MASK

mymodel=load_model('mymodel.h5')

cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=mymodel.predict(test_image)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
          
    cv2.imshow('img',img)
    
    if cv2.waitKey(1)==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()